In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import os

# Plot settings
%matplotlib inline
plt.style.use('ggplot')
fontsize = 20 # size for x and y ticks
plt.rcParams['legend.fontsize'] = fontsize
plt.rcParams.update({'font.size': fontsize})

In [2]:
# create a database connection
sqluser = 'postgres'
dbname = 'eicu'
schema_name = 'eicu_crd'
sqlhost = 'localhost'
sqlport = 5432
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, host=sqlhost, port=sqlport)

In [3]:
query_schema = 'set search_path to public,eicu_crd_phi;'
query = query_schema + """
with t1 as
(
select
    patientunitstayid
    , treatmentoffset
    , case
        when treatmentstring like 'pulmonary|ventilation and oxygenation|mechanical ventilation%' then 1
        when treatmentstring like 'surgery|pulmonary therapies|mechanical ventilation%' then 1
        when treatmentstring like 'toxicology|drug overdose|mechanical ventilation%' then 1
    else 0 end as mechvent
from treatment
)
select 
    patientunitstayid
    , min(treatmentoffset) as mvstart
    , max(treatmentoffset) as mvend
from t1
where mechvent = 1
group by patientunitstayid
order by patientunitstayid
"""

tr = pd.read_sql_query(query, con)

In [8]:
query_schema = 'set search_path to public,eicu_crd_phi;'
query = query_schema + """
select
    apv.patientunitstayid
    , oOBIntubDay1 as mv_apache
from apachepredvar apv
inner join apachepatientresult apr
on apv.patientunitstayid = apr.patientunitstayid
and apacheversion = 'IVa'
where apr.predictedhospitalmortality != '-1'
"""

ap = pd.read_sql_query(query, con)

In [16]:
# cross-reference
df = ap.merge(tr, how='left', on='patientunitstayid').copy()
df['mv_treatment'] = (df['mvstart'] < 1440).astype(int)

pd.crosstab(df['mv_apache'],df['mv_treatment'], margins=True, normalize=True)

mv_treatment,0,1,All
mv_apache,,,
0,0.726071,0.012853,0.738924
1,0.055536,0.205540,0.261076
All,0.781607,0.218393,1.000000


In [15]:
df.loc[~df['mvstart'].isnull(),:]

,patientunitstayid,mv_apache,mvstart,mvend,mv_treatment
4709,242154,1,204.0,4683.0,1
4732,243283,1,33.0,503.0,1
4735,243494,1,392.0,3851.0,1
4737,243643,1,77.0,142.0,1
4745,243990,1,103.0,125.0,1
4751,244255,0,-389.0,-306.0,1
4757,244445,1,184.0,3162.0,1
4763,244718,1,27.0,1134.0,1
4765,244763,0,993.0,1125.0,1
4767,244975,1,74.0,1601.0,1
